In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import imutils
from tqdm import tqdm

In [2]:
pics_root_path = '/Users/imrankabir/Desktop/research/bengali_ocr_app/bengali_ocr/data/'
pics_source_path = os.path.join(pics_root_path, 'acceptedImage')
image_dest_path = os.path.join(pics_root_path, 'cropped_image')

if not os.path.exists(image_dest_path):
    os.makedirs(image_dest_path)

In [3]:
image_names = os.listdir(pics_source_path)
for img_name in tqdm(image_names):
    # img_name = 'খীং.png'
    if not '.png' in img_name:
        continue
    img_path = os.path.join(pics_source_path, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    img = cv2.threshold(img, 45, 255, cv2.THRESH_BINARY)[1]
    inv_img = cv2.bitwise_not(img)
    im_blr = cv2.blur(inv_img, (40, 40))
    im_blr = cv2.GaussianBlur(im_blr, (5, 5), 0)
      
    thresh = cv2.threshold(im_blr, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    cnts = cv2.findContours(image=thresh, mode=cv2.RETR_EXTERNAL, 
                                           method=cv2.CHAIN_APPROX_SIMPLE)
    contours = cnts[0]
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    square_left_top = (extLeft[0], extTop[1])
    square_right_bot = (extRight[0], extBot[1])
    
    inv_img = inv_img[square_left_top[1]:square_right_bot[1], square_left_top[0]:square_right_bot[0]]
    
    cont_img = np.zeros(img.shape)
    cv2.drawContours(image=cont_img, contours=contours, 
                     contourIdx=-1, color=(255, 255, 255), thickness=2, 
                     lineType=cv2.LINE_AA)
    
    out_img = np.zeros((max(inv_img.shape), max(inv_img.shape)))
    
    if inv_img.shape[0] == out_img.shape[0]:
        devi = out_img.shape[1] - inv_img.shape[1]
        out_img[0:, devi//2:(out_img.shape[1]-(devi - devi//2))] = inv_img     
    elif inv_img.shape[1] == out_img.shape[1]:
        devi = out_img.shape[0] - inv_img.shape[0]
        out_img[devi//2:(out_img.shape[1]-(devi - devi//2)), 0:] = inv_img 
        
    out_img = cv2.resize(out_img, (224, 224), interpolation = cv2.INTER_AREA)
    out_img = cv2.threshold(out_img, 45, 255, cv2.THRESH_BINARY)[1]
    # print(np.unique(out_img))
    out_img = 255 - out_img
    
    save_pic_path = os.path.join(image_dest_path, img_name)
    cv2.imwrite(save_pic_path, out_img)
    # print(np.unique(out_img))
    # break
    
# plt.imshow(out_img)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10909/10909 [00:35<00:00, 303.85it/s]
